# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import numpy as np
import pandas as pd
import pickle
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sqlalchemy import create_engine

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import precision_recall_fscore_support
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/soohyeonkim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/soohyeonkim/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/soohyeonkim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///FigureEight_ETL.db')
df = pd.read_sql("FigureEight_ETL", con=engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
## Define feature and target variables X and Y
X = df['message'].values
## define classes names
categories = df.columns[4:].values
Y = df[categories].values
X.shape, Y.shape

((26216,), (26216, 36))

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    """
    Normalize, tokenize, lemmatize, clean texts
    
    > Parameters:
    text: raw text
    
    > Returns:
    clean_tokens: tokens that went through aformentioned procedures
    """
    
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
## get stopwords ready
stopwords = tokenize(" ".join(stopwords.words('english')))
stopwords.extend([str(i) for i in range(0, 1000)])
stopwords.extend(['000'])

## define random forest pipeline
rfc_pipe = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize, stop_words=stopwords)),
        ('tfidf', TfidfTransformer()),
        ('rfc', MultiOutputClassifier(RandomForestClassifier(n_estimators=50, n_jobs=10, random_state=4999)))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
%%time
## spliting train & test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=1004)

## fit pipeline classifier
rfc_pipe.fit(X_train, y_train)
y_pred = rfc_pipe.predict(X_test)

CPU times: user 4min 38s, sys: 1.94 s, total: 4min 40s
Wall time: 40.8 s


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
def multioutput_classification_report(y_true, y_pred):
    """
    This is classification report for multioutput classifiers
    
    > Parameters:
    y_true: true labels; numpy.ndarray
    y_pred: predicted labels; numpy.ndarray
    
    > Returns: None
    """
    
    supports = y_true.sum(axis=0)
    print("{:>24s}{:>12s}{:>12s}{:>12s}{:>12s}".format('', 'Precision', 'Recall', 'F1_score', 'support'))
    for i in range(0, y_true.shape[1]):
        _ = precision_recall_fscore_support(y_true[:, i], y_pred[:, i], average='micro')
        print("{:>24s}{:>12.2f}{:>12.2f}{:>12.2f}{:>12.2f}".format(categories[i], _[0], _[1], _[2], supports[i]))
        
multioutput_classification_report(y_test, y_pred)

                           Precision      Recall    F1_score     support
                 related        0.81        0.81        0.81     4998.00
                 request        0.89        0.89        0.89     1116.00
                   offer        1.00        1.00        1.00       27.00
             aid_related        0.78        0.78        0.78     2679.00
            medical_help        0.92        0.92        0.92      530.00
        medical_products        0.95        0.95        0.95      324.00
       search_and_rescue        0.97        0.97        0.97      181.00
                security        0.98        0.98        0.98      120.00
                military        0.97        0.97        0.97      210.00
             child_alone        1.00        1.00        1.00        0.00
                   water        0.96        0.96        0.96      399.00
                    food        0.95        0.95        0.95      702.00
                 shelter        0.94        0.94   

### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
# rfc_pipe.get_params()

In [9]:
parameters = {
#     'vect__ngram_range': [(1, 1), (1, 2)],
#     'vect__max_df': [0.8, 1.0],
    'rfc__estimator__n_estimators': [30, 50]
}

cv = GridSearchCV(rfc_pipe, param_grid=parameters)

In [10]:
%%time
cv.fit(X_train, y_train)

print(cv.best_params_)

{'rfc__estimator__n_estimators': 50}
CPU times: user 18min 12s, sys: 8.97 s, total: 18min 21s
Wall time: 3min 35s


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
y_pred = cv.predict(X_test)
multioutput_classification_report(y_test, y_pred)

                           Precision      Recall    F1_score     support
                 related        0.81        0.81        0.81     4998.00
                 request        0.89        0.89        0.89     1116.00
                   offer        1.00        1.00        1.00       27.00
             aid_related        0.78        0.78        0.78     2679.00
            medical_help        0.92        0.92        0.92      530.00
        medical_products        0.95        0.95        0.95      324.00
       search_and_rescue        0.97        0.97        0.97      181.00
                security        0.98        0.98        0.98      120.00
                military        0.97        0.97        0.97      210.00
             child_alone        1.00        1.00        1.00        0.00
                   water        0.96        0.96        0.96      399.00
                    food        0.95        0.95        0.95      702.00
                 shelter        0.94        0.94   

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [12]:
%%time
## adaboost pipeline
abc_pipe = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize, stop_words=stopwords)),
        ('tfidf', TfidfTransformer()),
        ('abc', MultiOutputClassifier(AdaBoostClassifier(n_estimators=50, random_state=9982)))
    ])

## parameters for adaboost pipeline
parameters_ada = {
#     'vect__ngram_range': [(1, 1), (1, 2)],
#     'vect__max_df': [0.8, 1.0],
#     'abc__estimator__n_estimators': [25, 50],
    'abc__estimator__learning_rate': [0.5, 1]
}

cv_ada = GridSearchCV(abc_pipe, param_grid=parameters_ada)
cv_ada.fit(X_train, y_train)
print(cv_ada.best_params_)

{'abc__estimator__learning_rate': 0.5}
CPU times: user 14min 55s, sys: 5.78 s, total: 15min
Wall time: 5min 16s


In [13]:
y_pred = cv_ada.predict(X_test)
multioutput_classification_report(y_test, y_pred)

                           Precision      Recall    F1_score     support
                 related        0.76        0.76        0.76     4998.00
                 request        0.89        0.89        0.89     1116.00
                   offer        1.00        1.00        1.00       27.00
             aid_related        0.75        0.75        0.75     2679.00
            medical_help        0.93        0.93        0.93      530.00
        medical_products        0.96        0.96        0.96      324.00
       search_and_rescue        0.97        0.97        0.97      181.00
                security        0.98        0.98        0.98      120.00
                military        0.97        0.97        0.97      210.00
             child_alone        1.00        1.00        1.00        0.00
                   water        0.97        0.97        0.97      399.00
                    food        0.95        0.95        0.95      702.00
                 shelter        0.95        0.95   

### 9. Export your model as a pickle file

In [14]:
import pickle
with open('classifier.pkl', 'wb') as file:
    pickle.dump(cv_ada, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [15]:
## tfidf matrix to achieve important words
tfidf = TfidfVectorizer(min_df=0.01, max_df=0.90, tokenizer=tokenize, stop_words=stopwords)
vect = CountVectorizer(tokenizer=tokenize, stop_words=stopwords)
tfidf.fit(X)
vect.fit(X)

tfidf_vocab = tfidf.vocabulary_
vect_vocab = vect.vocabulary_

## make tfidf dataframe - word & rank
tfidf_df = pd.DataFrame.from_dict(tfidf_vocab, orient='index').reset_index()
tfidf_df.columns = ['word', 'rank']

## make count vectorizer dataframe - word & count
vect_df = pd.DataFrame.from_dict(vect_vocab, orient='index').reset_index()
vect_df.columns = ['word', 'count']

## merge two dataframe in the order of count
vocab_df = pd.merge(tfidf_df, vect_df, how='left', on=['word']).sort_values('count', ascending=False).reset_index(drop=True)
vocab_df = vocab_df[['word', 'count']]

In [16]:
vocab_df.head(20)

,word,count
0,year,31038
1,would,30771
2,world,30743
3,working,30739
4,work,30733
5,woman,30699
6,without,30659
7,west,30423
8,well,30388
9,week,30361
